In [ ]:
pip install qiskit

In [ ]:
pip install qiskit-aer

In [ ]:
!pip install pylatexenc

In [ ]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.circuit.library import QFT , CCXGate, UnitaryGate
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
from qiskit.circuit.library import Diagonal
from qiskit.quantum_info import Operator

In [ ]:
pip install plotly==5.22.0

In [ ]:
pip install nbformat>=4.2.0



---



---



---



In [ ]:
# Function to normalize a vector
def normalize(coeffs):
    norm = np.linalg.norm(coeffs)
    return [c / norm for c in coeffs], norm

# for now this only works for same lenght norm array 1 and 2. fo unequal array lenghts we need to add padding.
norm_array1, norm1 = normalize([1,2,3,4, 5,6,7,8])
norm_array2, norm2 = normalize([ 5,6,7,8, 1,2,3,4])
print(norm_array1 + [0]*len(norm_array1))
print(norm_array2 + [0]*len(norm_array2))
print(norm_array1, norm_array2)
print(norm1, norm2)

[0.07001400420140048, 0.14002800840280097, 0.21004201260420147, 0.28005601680560194, 0.3500700210070024, 0.42008402520840293, 0.4900980294098034, 0.5601120336112039, 0, 0, 0, 0, 0, 0, 0, 0]
[0.3500700210070024, 0.42008402520840293, 0.4900980294098034, 0.5601120336112039, 0.07001400420140048, 0.14002800840280097, 0.21004201260420147, 0.28005601680560194, 0, 0, 0, 0, 0, 0, 0, 0]
[0.07001400420140048, 0.14002800840280097, 0.21004201260420147, 0.28005601680560194, 0.3500700210070024, 0.42008402520840293, 0.4900980294098034, 0.5601120336112039] [0.3500700210070024, 0.42008402520840293, 0.4900980294098034, 0.5601120336112039, 0.07001400420140048, 0.14002800840280097, 0.21004201260420147, 0.28005601680560194]
14.2828568570857 14.2828568570857


# classical part

In [ ]:
import numpy as np

# Define the two arrays
array1 = np.array(norm_array1 + [0]*len(norm_array1))
array2 = np.array(norm_array2 + [0]*len(norm_array2))

# Compute the element-wise product
elementwise_product = np.multiply(array1, array2)

# Print the result
print("Element-wise product:", elementwise_product)

Element-wise product: [0.0245098  0.05882353 0.10294118 0.15686275 0.0245098  0.05882353
 0.10294118 0.15686275 0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [ ]:
import plotly.graph_objects as go

# Perform convolution
convolution_result = np.convolve(array1, array2, mode='full')

traces = []
traces.append(go.Scatter(y=array1, name="input signal"))
traces.append(go.Scatter(y=convolution_result, name="filtered result"))
fig = go.Figure(traces)
fig.show()

# quantum part

In [ ]:
qubits = int(np.log2(len(array1)))
print(qubits)

4


In [ ]:
import numpy as np

def qft_matrix(n):
    """Generate the QFT matrix for n qubits (size 2^n x 2^n)."""
    N = 2**n
    omega = np.exp(2j * np.pi / N)
    qft_mat = np.zeros((N, N), dtype=complex)

    for i in range(N):
        for j in range(N):
            qft_mat[i, j] = omega**(i * j)

    return qft_mat / np.sqrt(N)

int(np.log2(len(array1)))
# Generate the QFT matrix for 4 qubits
qft_mat_4_qubits = qft_matrix(qubits)

# Print the QFT matrix
print("QFT Matrix for 3 Qubits:")
print(qft_mat_4_qubits)
print(np.shape(qft_mat_4_qubits))

QFT Matrix for 3 Qubits:
[[ 2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j]
 [ 2.50000000e-01+0.00000000e+00j  2.30969883e-01+9.56708581e-02j
   1.76776695e-01+1.76776695e-01j  9.56708581e-02+2.30969883e-01j
  -5.55111512e-17+2.50000000e-01j -9.56708581e-02+2.30969883e-01j
  -1.76776695e-01+1.76776695e-01j -2.30969883e-01+9.56708581e-02j
  -2.50000000e-01-1.11022302e-16j -2.30969883e-01-9.56708581e-02j
  -1.76776695e-01-1.76776695e-01j -9.56708581e-02-2.30969883e-01j
   1.66533454e-16-2.50000000e-01j  9.56708581e-02-

## step 1: qft matrix on the normalised filter array and window array

In [ ]:
# Define the filter array with 8 non-zero elements and 8 zeros
# filter = [0.0020939558974013665, 0.04493999659839265, 0.267769868918511, 0.5812076801643039,
#           0.6512469871508397, 0.39116353951309446, 0.10554509675346828, 0.014199348776285179] + [0]*8

# window= [0]*0 + [ 2.84459484e-01, 1.60769035e-01, 6.52830735e-01, 1.40302564e-01,
#   4.04440838e-01, 5.49152208e-02, 5.29882315e-01,  7.34968555e-16] + [0]*8

# Convert the filter to a numpy array
filter_array = np.array(array1)

# Perform matrix-vector multiplication
result1 = np.dot(qft_mat_4_qubits, filter_array)
result_array1 = np.array(result1)
# Print the resulting vector
print("Result of QFT matrix multiplied by filter array:")
print(result1)

Result of QFT matrix multiplied by filter array:
[ 0.63012604+0.00000000e+00j -0.14242723+4.39980210e-01j
 -0.070014  -1.69028758e-01j  0.05916333+1.30979203e-01j
 -0.070014  -7.00140042e-02j  0.07485842+5.84773275e-02j
 -0.070014  -2.90007501e-02j  0.07841948+1.74083142e-02j
 -0.070014  -2.48739710e-16j  0.07841948-1.74083142e-02j
 -0.070014  +2.90007501e-02j  0.07485842-5.84773275e-02j
 -0.070014  +7.00140042e-02j  0.05916333-1.30979203e-01j
 -0.070014  +1.69028758e-01j -0.14242723-4.39980210e-01j]


In [ ]:
# Define the filter array with 8 non-zero elements and 8 zeros
# filter = [0.0020939558974013665, 0.04493999659839265, 0.267769868918511, 0.5812076801643039,
#           0.6512469871508397, 0.39116353951309446, 0.10554509675346828, 0.014199348776285179] + [0]*8

# window= [0]*0 + [ 2.84459484e-01, 1.60769035e-01, 6.52830735e-01, 1.40302564e-01,
#   4.04440838e-01, 5.49152208e-02, 5.29882315e-01,  7.34968555e-16] + [0]*8

# Convert the filter to a numpy array
window_array = np.array(array2)

# Perform matrix-vector multiplication
result1w = np.dot(qft_mat_4_qubits, window_array)
result_array1w = np.array(result1w)
# Print the resulting vector
print("Result of QFT matrix multiplied by filter array:")
print(result1w)

Result of QFT matrix multiplied by filter array:
[ 0.63012604+0.00000000e+00j  0.20955693+3.69966206e-01j
  0.070014  +1.69028758e-01j -0.04562003+2.00993207e-01j
 -0.070014  -7.00140042e-02j  0.12164029-1.15366767e-02j
  0.070014  +2.90007501e-02j  0.06449283+8.74223184e-02j
 -0.070014  -2.48739710e-16j  0.06449283-8.74223184e-02j
  0.070014  -2.90007501e-02j  0.12164029+1.15366767e-02j
 -0.070014  +7.00140042e-02j -0.04562003-2.00993207e-01j
  0.070014  -1.69028758e-01j  0.20955693-3.69966206e-01j]


# step 2: element wise product of the qft matrix product* window array and qft matrix product* filter array

In [ ]:
# Perform element-wise multiplication
elementwise_product = np.multiply(result1, result1w)

# Alternatively, you can use the `*` operator for element-wise multiplication
# elementwise_product = array1 * array2

# Print the result
print("Element-wise product:", elementwise_product)

Element-wise product: [ 3.97058824e-01+0.00000000e+00j -1.92624424e-01+3.95076406e-02j
  2.36687604e-02-2.36687604e-02j -2.90249630e-02+5.91615258e-03j
 -3.12538984e-17+9.80392157e-03j  9.78043425e-03+6.24958142e-03j
 -4.06091728e-03-4.06091728e-03j  3.53561917e-03+7.97832436e-03j
  4.90196078e-03+3.48305263e-17j  3.53561917e-03-7.97832436e-03j
 -4.06091728e-03+4.06091728e-03j  9.78043425e-03-6.24958142e-03j
  1.08927306e-16-9.80392157e-03j -2.90249630e-02-5.91615258e-03j
  2.36687604e-02+2.36687604e-02j -1.92624424e-01-3.95076406e-02j]


# step 3: IQFT acts on the resulting staevector from previous operations

In [ ]:
import numpy as np

def inverse_qft_matrix(n):
    """Generate the inverse QFT matrix for n qubits (size 2^n x 2^n)."""
    N = 2**n
    omega = np.exp(-2j * np.pi / N)  # Use the inverse root of unity
    iqft_mat = np.zeros((N, N), dtype=complex)

    for i in range(N):
        for j in range(N):
            iqft_mat[i, j] = omega**(i * j)

    return iqft_mat / np.sqrt(N)

# Generate the inverse QFT matrix for 4 qubits
iqft_mat_4_qubits = inverse_qft_matrix(qubits)

# Print the inverse QFT matrix
print("Inverse QFT Matrix for 3 Qubits:")
print(iqft_mat_4_qubits)

Inverse QFT Matrix for 3 Qubits:
[[ 2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j
   2.50000000e-01+0.00000000e+00j  2.50000000e-01+0.00000000e+00j]
 [ 2.50000000e-01+0.00000000e+00j  2.30969883e-01-9.56708581e-02j
   1.76776695e-01-1.76776695e-01j  9.56708581e-02-2.30969883e-01j
  -5.55111512e-17-2.50000000e-01j -9.56708581e-02-2.30969883e-01j
  -1.76776695e-01-1.76776695e-01j -2.30969883e-01-9.56708581e-02j
  -2.50000000e-01+1.11022302e-16j -2.30969883e-01+9.56708581e-02j
  -1.76776695e-01+1.76776695e-01j -9.56708581e-02+2.30969883e-01j
   1.66533454e-16+2.50000000e-01j  9.56708

In [ ]:
# Perform matrix-vector multiplication
final_result = np.dot(iqft_mat_4_qubits,  elementwise_product)

# Print the resulting vector
print("Result of QFT matrix multiplied by filter array:")
print(final_result)

Result of QFT matrix multiplied by filter array:
[6.12745098e-03-3.73832909e-16j 1.96078431e-02-3.42802840e-16j
 4.16666667e-02-2.85235001e-16j 7.35294118e-02-2.26835956e-16j
 1.06617647e-01-1.76125454e-16j 1.42156863e-01-1.15961743e-16j
 1.81372549e-01-4.76775028e-17j 2.25490196e-01-6.20607839e-17j
 2.14460784e-01+3.06168265e-16j 1.86274510e-01+2.39543094e-16j
 1.39705882e-01+1.25354760e-16j 7.35294118e-02+7.22774922e-17j
 7.47549020e-02+1.49789679e-16j 6.37254902e-02+4.13411290e-17j
 3.92156863e-02+3.61866597e-16j 1.38777878e-17+5.43268031e-17j]


In [ ]:
traces = []
traces.append(go.Scatter(y=array1, name="input signal"))
traces.append(go.Scatter(y=np.real(final_result)*np.sqrt(len(array1)), name="quantum filtered signal")) # explain why we have 2*sqrt(2) for rescaling here. explain why len[A] is in here????
# traces.append(go.Scatter(y=convolution_result, name="numpy filtered result"))
fig = go.Figure(traces)
fig.show()

traces = []
traces.append(go.Scatter(y=array1, name="input signal"))
# traces.append(go.Scatter(y=np.real(final_result)*len(A)*2*np.sqrt(2), name="quantum filtered signal"))
traces.append(go.Scatter(y=convolution_result[:len(array1)], name="numpy filtered result"))
fig = go.Figure(traces)
fig.show()

traces = []
# traces.append(go.Scatter(y=array1, name="input signal"))
traces.append(go.Scatter(y=np.real(final_result)*np.sqrt(len(array1)), name="quantum filtered signal"))
traces.append(go.Scatter(y=convolution_result, name="numpy filtered result"))
fig = go.Figure(traces)
fig.show()

In [ ]:
scaling = np.real(final_result*np.sqrt(len(array1)))[:5]/ convolution_result[:5]
print(scaling)

[1. 1. 1. 1. 1.]




---



---



---



In [ ]:
import numpy as np

def cnot_matrix(num_qubits, control_qubit, target_qubit):
    dim = 2 ** num_qubits
    cnot_mat = np.eye(dim, dtype=complex)

    for i in range(dim):
        binary_state = np.binary_repr(i, width=num_qubits)
        control_bit = int(binary_state[control_qubit])
        target_bit = int(binary_state[target_qubit])

        if control_bit == 1:
            flipped_state = binary_state[:target_qubit] + str(1 - target_bit) + binary_state[target_qubit+1:]
            flipped_index = int(flipped_state, 2)
            cnot_mat[i, i] = 0
            cnot_mat[i, flipped_index] = 1

    return cnot_mat

# Example usage
num_qubits = 4
control_qubit = 0  # 1st qubit (0-based index)
target_qubit = 2   # 3rd qubit (0-based index)

cnot_mat = cnot_matrix(num_qubits, control_qubit, target_qubit)
print("CNOT Matrix for control qubit 0 and target qubit 2:")
print(cnot_mat)


CNOT Matrix for control qubit 0 and target qubit 2:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+